<a href="https://colab.research.google.com/github/anandhc6/Assignment1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

#Data preperation

In [1]:
import numpy as np
from numpy import random
from keras.datasets import fashion_mnist
#import the required libraries
from matplotlib import pyplot
import math as math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

(X, y), (X_test, y_test) = fashion_mnist.load_data()

# Reshaping the data 
X = X.reshape(X.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

# Normalizing
X = X/255.0
X_test = X_test/255.0

# Split the data X into a training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

#Layer dimensions
la =[784,256,128,10]

L=len(la)-1

#weights and bias empty initializing
np.random.seed(42)
params={}


4431872/4422102 [==============================] - 0s 0us/step
